# COMP9517: Computer Vision, 2020 Term 2
## Group Project 
### Detecting and Tracking of Cells in Time-Lapse Microscopy


In [30]:
# Import Libraries 
import os
from matplotlib import pyplot as plt
from math import sqrt
import cv2 as cv

# Import Detector
from preprocessing.preprocess import *
from preprocessing.model import *
from detection import *

# Import Tracker
from tracker import *

# Mouse event detection
mouseX = mouseY = 0

def detect_mouse(event,x,y,flags,param):
    global mouseX, mouseY
    if event == cv.EVENT_LBUTTONDOWN:
        mouseX,mouseY = x,y

### User Setup
The directory for this notebook should include the:
- This notebook 'Group_Project.ipynb'
- The sub-directory preprocessing, this includes the Unet Model used for detection, the preprocessing steps and the saved model weights
- The detection.py and tracker.py file
- The sub-directory 'inputs' that includes the files extracted from the 'COMP9517 20T2 Group Project Image Sequences v2.zip'

### User inputs
Set the f_flag value to the required data set.
0 - DIC-C2DH-HeLa
1 - Fluo-N2DL-HeLa
2 - PhC-C2DL-PSC

and sequence to the sequence number required. 

In [32]:
# Global parameters
f_flag = 2
sequence = 1  

if f_flag==0:
    path = f"input/DIC-C2DH-HeLa/Sequence {sequence}"
elif f_flag==1:
    path = f"input/Fluo-N2DL-HeLa/Sequence {sequence}"
else:
    path = f"input/PhC-C2DL-PSC/Sequence {sequence}"
    
# Image sequences 
im_files = [f for f in os.listdir(path) if f[-3:] == 'tif']
print("Number of files: %d\nFirst File: %s" % (len(im_files), im_files[0]))   

Number of files: 426
First File: t000.tif


## Task 1 Detect and Track Cells
### 1.1 Cell Detection
1. Preprocess training data and store in .../training
2. Augment training data and feed into unet model
3. Train model
4. Segment cells using unet for DIC-C2DH-HeLa
5. Use other preprocessing techniques for Fluo-N2DL-HeLa and PhC-C2DL-PSC
6. Draw bounding boxes for all images

In [34]:
# Create and preprocess training images
# need to create the folders beforehand, if not it won't write
# training images are stored in {path}/training/image
# masks are stored in {path}/training/image 
create_training(path, f_flag)

In [36]:
# U-Net model
model = unet()

# Load previously trained model
model.load_weights('preprocessing/unet_model.hdf5')

0.001


### 1.2 Cell Tracking

CentroidTracker, is a simple tracking algorithm that takes the input of a list of bounding boxes and calculates the centroid (cX,cY), and tries to match the closes object in the next frame. The output is a list of object centroids and the trajectory based on previous frames. 

ExtTracker, is an extension of Centroid Tracker that will calculate the nearest neighbour based on the centroid and 3 additional features, these features have and adjustable weight that can be set when initialising to prioritise particular features. 

In [38]:
# Initalise tracker
tracker = CentroidTracker()
trackerType = 0
# initalise boxes input "boxes" list of bounding boxes (startX, startY, endX, endY)
boxes = []

In [40]:
# Initalise tracker with weights for each feature
tracker = ExtTracker(alpha1 = 0.01, alpha2 = 0.01, alpha3 = 0.01)
trackerType = 1
# initalise boxes input "shapes" list of bounding boxes and features (startX, startY, endX, endY, Feature1, Feature2, Feature3)
shapes = []

### 1.3 Analysing Cell Motion
We can calculate the speed, total distance, net distance and subsequent confinement ratio of each cell by simply left-clicking on the selected cell and then pressing the 'a' key on your keyboard. The program should then print out the results everytime the 'a' key is pressed. To select a new cell, left-click on the new cell and press the 'a' key again. 


In [45]:
for f in im_files:
    #Read each file in and copy to frame for processing and output
    im = cv.imread("%s/%s" % (path, f))
    frame = im.copy()
    
    # detect the cells and features from the frame 
    # 'contours' is the sell outline used for motosis detection and feature extraction
    # 'boxes' is a list to bounding boxes fro each cell in the frame
    contours, boxes = detect(frame, f_flag, model)
    frame = c_stretch(frame, 0, 255)
    
    features = get_features(frame, contours)

    text = "Number of Cells Detected: {}".format(len(boxes))
    cv.putText(frame, text, (20, 20), cv.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1)
    
    # cumulative distribution function used for mitosis deection 
    cdf = get_cdf(frame, f_flag)
    mitosis_count = 0
    
    for cnt in contours:
        if (is_mitosis(frame, f_flag, cdf, cnt)):
            colour = (0, 255, 0)
            mitosis_count += 1
        else:
            colour = (0, 255, 255)
        x, y, w, h = cv.boundingRect(cnt)
        cv.rectangle(frame, (x, y), (x+w, y+h), colour, 1)
        
    text = "Number of Mitoses Detected: {}".format(mitosis_count)
    cv.putText(frame, text, (20, 40), cv.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 1)
    
    if trackerType: 
        #If extended tracker input boxes needs to be in the form, [(x1, y1, x2, y2, F1, F2, F3)]
        # Place holder features 
        objects, trajectorys = tracker.update(features)
    else: 
        #If Centroid Tracker 
        objects, trajectorys = tracker.update(boxes)
    
    # loop over the tracked objects, trajectory is a list of all the previous object centres [(cXn-1, cYn-1), (cXn, cYn)]
    for (objectID, trajectory) in trajectorys.items():
        if len(trajectory) > 1:            
            for i in range(1, len(trajectory)):
                x1y1 = (trajectory[i-1][0], trajectory[i-1][1])
                x2y2 = (trajectory[i][0], trajectory[i][1])
                cv.line(frame, x1y1, x2y2, (255, 0, 0), 1)

    # show the output frame
    cv.imshow("Frame", frame)
    cv.setMouseCallback('Frame', detect_mouse)
    k = cv.waitKey(1)
    # pressing the esc key will break the loop
    if k == 27:
        break
    # left clicking anywhere in the image and pressing 'a' will print the coordinates and the analysys of the nearest cell
    elif k == ord('a'):
        print(f"Clicked at: ({mouseX}, {mouseY})")
        Speed, TotalDistance, NetDistance, ConfinementRatio = tracker.analyse((mouseX, mouseY))
        print("Speed: %.3f pixels per frame\nTotalDistance: %.3f pixels\nNetDistance: %.3f pixels\nConfinementRatio: %.3f\n" %
              (Speed, TotalDistance, NetDistance, ConfinementRatio))

cv.destroyAllWindows()

In [43]:
#Debug to force close window on error
cv.destroyAllWindows()

Model develpment and training for reference

In [44]:
# Part of U-net training section
## Additional parameters to mess around for augmentation:
target_size = (256,256)
batch_size = 32
rotation_range = 30
zoom_range = 0.15
width_shift_range = 0.2
height_shift_range = 0.2
shear_range = 0.15

## Augment training data
training_generator = training_augmentation(path, target_size, batch_size, rotation_range, zoom_range, 
                                            width_shift_range, height_shift_range, shear_range)
# train new model
# model_checkpoint = ModelCheckpoint(
#     'unet_model1.hdf5',
#     monitor='loss',
#     save_best_only=True
# )

# import tensorflow as tf
# with tf.device("cpu:0"):
#     model.fit(training_generator, steps_per_epoch=300, epochs=2, callbacks=[model_checkpoint])

# U-Net model
model = unet()

# or load previously trained model
model.load_weights('preprocessing/unet_model.hdf5') # This is still funky, i gotta touch up on the unet

# predict mask, draw bounding box and output final images
# output is stored in output/{path}/Sequence [1-4]/txxx.tif
# make sure the folder exists in your computer first, 
# idk why it doesn't auto create it
cell_detection(path, f_flag, model)

Found 0 images belonging to 1 classes.
Found 0 images belonging to 1 classes.
0.001
